# Encontrando insights de la UEFA Champions League

La Liga de Campeones de la UEFA, a menudo llamada Liga de Campeones, es una competencia anual de fútbol por excelencia que cautiva a los fanáticos de todo el mundo. Establecida en 1955 como la Copa de Clubes Campeones de Europa, evolucionó a la Liga de Campeones de la UEFA en 1992, ampliando su atractivo. El formato moderno cuenta con 32 equipos de clubes de primer nivel seleccionados en función de su desempeño en la liga nacional, lo que aumenta la intriga.  

Este evento electrizante trasciende los deportes y se convierte en una celebración de la unidad, la cultura y el orgullo nacional. Los fanáticos, vestidos con los colores de sus países, crean una atmósfera eléctrica, lo que hace que el torneo sea tanto sobre los espectadores como sobre los jugadores. Financieramente, la Liga de Campeones es un salvavidas para los clubes, ya que aumenta los ingresos y ofrece oportunidades transformadoras. Sin embargo, genera debates sobre las disparidades de riqueza en el fútbol europeo.  

La Liga de Campeones es sinónimo de rivalidades históricas, triunfos de los desvalidos y brillantez individual. Para los jugadores, representa la cima de su carrera, mientras que para los fanáticos, es un fenómeno cultural. El himno y los rituales icónicos enriquecen la experiencia futbolística. En 200 palabras, la UEFA Champions League es el epítome de la excelencia del fútbol europeo, que ofrece momentos inolvidables, recompensas económicas y un impacto cultural único, con 32 clubes de primer nivel que se suman a su atractivo.

**Nombres de las tablas o ficheros a utilizar**:  
[uefa_2020.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/EQuW8d16tZJKpYZN6jR480sBgc1IYypAw9hGHKQARS560g?e=7TybBU)  
[uefa_2021.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ESG8YBrjD3tBky3aGT0MwIABNt_PQxFVVYVlWMnZYUmV8g?e=VrYHtX)  
[uefa_2022.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/Ecf8O8U8sxJLoW9Bkd2ZVzgBb_gZc35mNcprnR9FMlzliQ?e=Vn3a6v)  
- Todas las tablas tienen las mismas columnas y data types

| Column | Definition | Data type |
|--------|------------|-----------|
| `STAGE`| Stage of the March | `VARCHAR(50)` |
| `DATE` | When the match occurred. | `DATE` |
| `PENS` | Did the match end with penalty | `VARCHAR(50)` |
| `PENS_HOME_SCORE` | In case of penalty, score by home team | `VARCHAR(50)` |
| `PENS_AWAY_SCORE` | In case of penalty, score by away team | `VARCHAR(50)` |
| `TEAM_NAME_HOME` | Team home name | `VARCHAR(50)` |
| `TEAM_NAME_AWAY`| Team away  name | `VARCHAR(50)` |
| `TEAM_HOME_SCORE` | Team home score | `NUMBER` |
| `TEAM_AWAY_SCORE` | Team away score | `NUMBER` |
| `POSSESSION_HOME` | Ball possession for the home team | `FLOAT` |
| `POSSESSION_AWAY` | Ball possession for the away team | `FLOAT` |
| `TOTAL_SHOTS_HOME` | Number of shots by the home team | `NUMBER` |
| `TOTAL_SHOTS_AWAY` | Number of shots by the away team | `NUMBER`
| `SHOTS_ON_TARGET_HOME` | Total shot for home team | `FLOAT` |
| `SHOTS_ON_TARGET_AWAY` | Total shot for away team | `FLOAT` |
| `DUELS_WON_HOME` | duel win possession of ball - for home team | `NUMBER` |
| `DUELS_WON_AWAY` | duel win possession of ball - for away team | `NUMBER` 
| `PREDICTION_TEAM_HOME_WIN` | Probability of home team to win | `FLOAT` |
| `PREDICTION_DRAW` | Probability of draw | `FLOAT` |
| `PREDICTION_TEAM_AWAY_WIN` | Probability of away team to win | `FLOAT` |
| `LOCATION` | Stadium where the match was held | `VARCHAR(50)` |

### Requerimientos:  
-Puedes utilizar SQL o PySpark

1. Encuentra los 3 equipos que anotaron más goles jugando en su estadio en la UEFA Champions League 2020-21. El resultado debe contener dos columnas: TEAM_NAME_HOME y TEAM_HOME_SCORE ordenadas en orden descendente de TEAM_HOME_SCORE. Guarda la consulta (o el notebook) como TEAM_HOME_WITH_MOST_GOALS.

In [0]:
from pyspark.sql.functions import sum
df1 = spark.read.option("header", "true").csv('dbfs:/FileStore/uefa_2020.csv')

df1 = df1.groupBy("TEAM_NAME_HOME").agg(sum("TEAM_HOME_SCORE").alias("goles_casa")).orderBy("goles_casa", ascending=False).limit(3)
df1.display()

TEAM_NAME_HOME,goles_casa
Manchester City,15.0
Real Madrid,14.0
Bayern Munich,13.0


Aqui agrupamos por partidos en casa y sumamos los goles con un alias. Este alias lo ordenamos de forma ascending false para que nos muestre de mayor a menor

2. Encuentra el equipo con posesión mayoritaria la mayor cantidad de veces durante la UEFA Champions League 2021-22. El resultado debe incluir dos columnas: TEAM_NAME y GAME_COUNT, que es la cantidad de veces que el equipo tuvo posesión mayoritaria durante un partido de fútbol. Guarda esta consulta (o el notebook) como TEAM_WITH_MAJORITY_POSSESSION

In [0]:
from pyspark.sql.functions import col, count,when, desc
df2 = spark.read.option("header", "true").csv('dbfs:/FileStore/uefa_2021.csv')
df2.display()

# Paso 1: Determinamos que equipo obtuvo una posesion mayor creando una nueva columna
df_2 = df2.withColumn("TEAM_NAME",
    when(col("POSSESSION_HOME") > col("POSSESSION_AWAY"), col("TEAM_NAME_HOME"))
    .otherwise(when(col("POSSESSION_AWAY") > col("POSSESSION_HOME"), col("TEAM_NAME_AWAY")))
)

# Filtrar los equipos que tuvieron posesión mayoritaria
df_posesion = df_2.filter(col("TEAM_NAME").isNotNull())

# Contar la cantidad de veces que cada equipo tuvo posesión mayoritaria
resultado = df_posesion.groupBy("TEAM_NAME").agg(count("TEAM_NAME").alias("GAME_COUNT")).orderBy(desc("GAME_COUNT")).limit(1)  # Ordenar por cantidad de juegos en posesión mayoritaria

resultado.display()

STAGE,DATE,PENS,PENS_HOME_SCORE,PENS_AWAY_SCORE,TEAM_NAME_HOME,TEAM_NAME_AWAY,TEAM_HOME_SCORE,TEAM_AWAY_SCORE,POSSESSION_HOME,POSSESSION_AWAY,TOTAL_SHOTS_HOME,TOTAL_SHOTS_AWAY,SHOTS_ON_TARGET_HOME,SHOTS_ON_TARGET_AWAY,DUELS_WON_HOME,DUELS_WON_AWAY,PREDICTION_TEAM_HOME_WIN,PREDICTION_DRAW,PREDICTION_TEAM_AWAY_WIN,LOCATION
Final,29-05-22,FALSE,FALSE,FALSE,Liverpool,Real Madrid,0,1,50%,50%,23,3,15,2,37%,63%,47%,2,53%,Anfield
Semi-finals second leg,05-05-22,FALSE,FALSE,FALSE,Real Madrid,Man City,3,1,44%,56%,14,15,5,10,47%,53%,49%,10,51%,Santiago Bernabeu
Semi-finals second leg,04-05-22,FALSE,FALSE,FALSE,Villarreal,Liverpool,2,3,44%,56%,6,15,2,5,49%,51%,49%,4,51%,El Madrigal
Semi-finals first leg,28-04-22,FALSE,FALSE,FALSE,Liverpool,Villarreal,2,0,74%,26%,19,1,5,0,41%,59%,50%,4,50%,Anfield
Semi-finals first leg,27-04-22,FALSE,FALSE,FALSE,Man City,Real Madrid,4,3,60%,40%,16,11,6,5,46%,54%,48%,5,52%,Etihad
Quarter-finals second leg,14-04-22,FALSE,FALSE,FALSE,Atl�tico de Madrid,Man City,0,0,39%,61%,14,10,3,1,49%,51%,45%,8,55%,Civitas Metropolitan Stadium
Quarter-finals second leg,14-04-22,FALSE,FALSE,FALSE,Liverpool,Benfica,3,3,67%,33%,17,6,6,4,38%,62%,45%,9,55%,Anfield
Quarter-finals second leg,13-04-22,FALSE,FALSE,FALSE,Real Madrid,Chelsea,2,3,43%,57%,10,28,4,7,37%,63%,40%,4,60%,Santiago Bernabeu
Quarter-finals second leg,13-04-22,FALSE,FALSE,FALSE,Bayern,Villarreal,1,1,68%,32%,23,4,4,1,38%,62%,38%,7,62%,Allianz Arena
Quarter-finals first leg,07-04-22,FALSE,FALSE,FALSE,Villarreal,Bayern,1,0,38%,62%,12,22,1,4,38%,62%,47%,5,53%,El Madrigal


TEAM_NAME,GAME_COUNT
Liverpool,9


3. Encuentra la lista de equipos de cada fase del juego que ganaron el duelo en un partido pero terminaron perdiendo el juego en el Campeonato de la UEFA 2022-23. El resultado debe contener dos columnas: STAGE y TEAM_LOST. Guarda la consulta (o el notebook) como TEAM_WON_DUEL_LOST_GAME_STAGE_WISE.

In [0]:
from pyspark.sql.functions import col, when

df3 = spark.read.option("header", "true").csv('dbfs:/FileStore/uefa_2022.csv')

df3.display()

# Filtrar los equipos que ganaron el duelo pero perdieron el partido y luego selecionar la fase y el equipo que perdio
result_df = df3.filter(
    ((col("DUELS_WON_HOME") > col("DUELS_WON_AWAY")) & (col("TEAM_HOME_SCORE") < col("TEAM_AWAY_SCORE"))) |
    ((col("DUELS_WON_AWAY") > col("DUELS_WON_HOME")) & (col("TEAM_AWAY_SCORE") < col("TEAM_HOME_SCORE")))
).select("STAGE",when(col("TEAM_HOME_SCORE") < col("TEAM_AWAY_SCORE"), col("TEAM_NAME_HOME")).otherwise(col("TEAM_NAME_AWAY")).alias("TEAM_LOST")
)

# Mostrar el resultado
result_df.display()


STAGE,DATE,PENS,PENS_HOME_SCORE,PENS_AWAY_SCORE,TEAM_NAME_HOME,TEAM_NAME_AWAY,TEAM_HOME_SCORE,TEAM_AWAY_SCORE,POSSESSION_HOME,POSSESSION_AWAY,TOTAL_SHOTS_HOME,TOTAL_SHOTS_AWAY,SHOTS_ON_TARGET_HOME,SHOTS_ON_TARGET_AWAY,DUELS_WON_HOME,DUELS_WON_AWAY,PREDICTION_TEAM_HOME_WIN,PREDICTION_DRAW,PREDICTION_TEAM_AWAY_WIN,LOCATION
Group stage: Matchday 1,6-9-2022,FALSE,FALSE,FALSE,Dinamo Zagreb,Chelsea,1,0,31%,69%,6,15,3,3,44%,56%,49%,10,51%,Maksimir
Group stage: Matchday 1,6-9-2022,FALSE,FALSE,FALSE,Salzburg,AC Milan,1,1,34%,66%,16,14,4,3,38%,62%,41%,9,59%,Red Bull Arena
Group stage: Matchday 1,6-9-2022,FALSE,FALSE,FALSE,Celtic,Real Madrid,0,3,35%,65%,10,12,4,6,46%,54%,47%,8,53%,Celtic Park
Group stage: Matchday 1,6-9-2022,FALSE,FALSE,FALSE,RB Leipzig,Shakhtar Donetsk,1,4,68%,32%,13,5,2,4,47%,53%,49%,8,51%,Red Bull Arena
Group stage: Matchday 1,6-9-2022,FALSE,FALSE,FALSE,Borussia Dortmund,K�benhavn,3,0,63%,37%,15,7,4,2,49%,51%,45%,6,55%,Signal Iduna Park
Group stage: Matchday 1,6-9-2022,FALSE,FALSE,FALSE,Sevilla,Manchester City,0,4,38%,62%,6,24,1,10,42%,58%,43%,2,57%,Ram�n S�nchez Pizju�n
Group stage: Matchday 1,6-9-2022,FALSE,FALSE,FALSE,Paris Saint-Germain,Juventus,2,1,58%,42%,15,13,6,4,36%,64%,45%,3,55%,Parc des Princes
Group stage: Matchday 1,6-9-2022,FALSE,FALSE,FALSE,Benfica,Maccabi Haifa,2,0,57%,43%,14,7,5,2,46%,54%,42%,8,58%,Stadion da Luz
Group stage: Matchday 1,7-9-2022,FALSE,FALSE,FALSE,Ajax,Rangers,4,0,76%,24%,17,4,9,1,47%,53%,45%,3,55%,Amsterdam Arena
Group stage: Matchday 1,7-9-2022,FALSE,FALSE,FALSE,Napoli,Liverpool,4,1,38%,62%,18,15,9,7,44%,56%,45%,8,55%,San Paolo


STAGE,TEAM_LOST
Group stage: Matchday 1,Chelsea
Group stage: Matchday 1,K�benhavn
Group stage: Matchday 1,Juventus
Group stage: Matchday 1,Maccabi Haifa
Group stage: Matchday 1,Rangers
Group stage: Matchday 1,Liverpool
Group stage: Matchday 1,Porto
Group stage: Matchday 1,Bayer Leverkusen
Group stage: Matchday 1,Viktoria Plze?
Group stage: Matchday 1,Marseille


Encontrar donde el quipo gane en duelo pero pierda ese partido. Para ello filtro por los partidos donde gana en duelos y pierde el partido comprobando ya sea de local o ya sea de visitante 